# Majumdar-Ghosh Model

The hamiltonian for the 1D Majumdar-Ghosh model is give by

\begin{equation}
H = \frac{J}{2}\sum_{i=1}^{N}\big\{\sigma_i^x\sigma_{i+1}^x+\sigma_i^y\sigma_{i+1}^y+\sigma_i^z\sigma_{i+1}^z\big\}+\frac{J}{4}\sum_{i=1}^{N}\big\{\sigma_i^x\sigma_{i+2}^x+\sigma_i^y\sigma_{i+2}^y+\sigma_i^z\sigma_{i+2}^z\big\}
\end{equation}

where $J$ is the coupling parameter and $\sigma^{x,y,z}$ are the Pauli operators <a name="ref-1"/>[(Majumdar, 1970)](#cite-Majumdar1970). In this example, the Variational Quantum Eigensolver method is used to find the approximate ground state energy of the above hamiltonian with boundary conditions $\sigma_{N+1} = \sigma_{1}$, and $\sigma_{N+2} = \sigma_{2}$. Note that the exact ground state energy is $-3NJ/4$ <a name="ref-2"/>[(Majumdar, 1970)](#cite-Majumdar1970). Implementation using __[Pennylane](https://pennylane.ai/)__.

In [1]:
import pennylane as qml
from scipy.optimize import minimize
from pennylane import numpy as np #Always import numpy from pennylane
import matplotlib.pyplot as plt

In [2]:
N = 4 #Number of spins
NS = 2**N #Number of states

#Quantum circuit for the first part of the hamiltonian
#Construct a QNode
#Initialize device
qcdev = qml.device("default.qubit", wires=N)
@qml.qnode(qcdev)
def Circuit1(params,coor,Wires):

    #Prepare state
    #Need to normalize parameters
    mag = np.sqrt(np.dot(params.T,params))
    params = params/mag
    qml.QubitStateVector(params, wires=range(N))
     
    indx = Wires[0]
    if coor == 0:
        if indx < N - 1:
            return qml.expval(qml.PauliX(indx) @ qml.PauliX(indx+1))
        else:
            return qml.expval(qml.PauliX(indx) @ qml.PauliX(0))
    
    elif coor == 1:
        if indx < N - 1:
            return qml.expval(qml.PauliY(indx) @ qml.PauliY(indx+1))
        else:
            return qml.expval(qml.PauliY(indx) @ qml.PauliY(0))
        
    else:
        if indx < N - 1:
            return qml.expval(qml.PauliZ(indx) @ qml.PauliZ(indx+1))
        else:
            return qml.expval(qml.PauliZ(indx) @ qml.PauliZ(0))

        
#Test the Circuit function
theta = np.random.rand(NS)
wires = [3]
print(Circuit1(theta,0,Wires=wires))

0.6159288262743123


In [3]:
#Quantum circuit for the second part of the hamiltonian
@qml.qnode(qcdev)
def Circuit2(params,coor,Wires):

    #Prepare state
    #Need to normalize parameters
    mag = np.sqrt(np.dot(params.T,params))
    params = params/mag
    qml.QubitStateVector(params, wires=range(N))
     
    indx = Wires[0]
    if coor == 0:
        if indx < N - 2:
            return qml.expval(qml.PauliX(indx) @ qml.PauliX(indx+2))
        else:
            return qml.expval(qml.PauliX(indx) @ qml.PauliX(indx+2-N))
    
    elif coor == 1:
        if indx < N - 2:
            return qml.expval(qml.PauliY(indx) @ qml.PauliY(indx+2))
        else:
            return qml.expval(qml.PauliY(indx) @ qml.PauliY(indx+2-N))
        
    else:
        if indx < N - 2:
            return qml.expval(qml.PauliZ(indx) @ qml.PauliZ(indx+2))
        else:
            return qml.expval(qml.PauliZ(indx) @ qml.PauliZ(indx+2-N))

        
#Test the Circuit function
vec = np.random.rand(NS)
wires = [3]
print(Circuit2(vec,2,Wires=wires))

0.3881528176426987


In [4]:
#Objective function to optimize
#This function return <H>/N
def Cost(theta,J):

    exp = 0.0
    for i in range(N):
        wrs = [i]
        for k in range(3):
            exp += Circuit1(theta,k,Wires=wrs)
            exp += 0.5*Circuit2(theta,k,Wires=wrs)
            
    return 0.5*J*exp/N 

#Test the Cost function
theta = np.random.rand(NS)
print(Cost(theta,1.0))

0.6068990070638836


In [5]:
#Optimization using COBYLA
J = 2 #Coupling parameter
Eg_exact = -1.5 #Exact value of ground state energy per site for J = 2

Theta0 = 0.01*np.random.rand(NS) #Initial guess for theta
Result = minimize(Cost, Theta0, args=(J), method='COBYLA', constraints=(), tol=1e-9)
print('Final parameters:')
print(Result.x)
Eg_sim = Cost(Result.x,J)
print("\n")
print("Eg_sim/N = %0.3f, Eg_exact/N = %0.3f\n"%(Eg_sim,Eg_exact))

Final parameters:
[ 9.37714088e-09 -7.98764073e-08  1.16493558e-07  2.14536342e+00
 -2.30595145e-09 -1.24408892e+00 -9.01274493e-01 -3.59505072e-09
 -3.54656873e-08 -9.01274417e-01 -1.24408890e+00  5.46459529e-08
  2.14536331e+00 -7.21792468e-08  4.97678696e-09  8.70746612e-09]


Eg_sim/N = -1.500, Eg_exact/N = -1.500



<!--bibtex

@article{Barmettler2010,
	doi = {10.1088/1367-2630/12/5/055017},
	url = {https://doi.org/10.1088/1367-2630/12/5/055017},
	year = 2010,
	month = {may},
	publisher = {{IOP} Publishing},
	volume = {12},
	number = {5},
	pages = {055017},
	author = {Barmettler, Peter and Matthias Punk and Vladimir Gritsev and Eugene Demler and Ehud Altman},
    title = {Quantum quenches in the anisotropic spin-$\frac{1}{2}$ Heisenberg chain: different approaches to many-body dynamics far from equilibrium},
	journal = {New Journal of Physics},
}

@book{MikeIke,
    title = {Quantum Computation and Quantum Information},
    author = {Nielsen, Michael A. and Chuang, Issac L},
    year = {2010},
    publisher = {Cambridge University Press}
}

@article{Majumdar1970,
	doi = {10.1088/0022-3719/3/4/019},
	url = {https://doi.org/10.1088/0022-3719/3/4/019},
	year = 1970,
	month = {apr},
	publisher = {{IOP} Publishing},
	volume = {3},
	number = {4},
	pages = {911--915},
	author = {Majumdar, C, K },
	title = {Antiferromagnetic model with known ground state},
	journal = {Journal of Physics C: Solid State Physics},
	abstract = {The ground state of the one-dimensional antiferromagnetic model, with the Hamiltonian H=1/2 j Sigma i=1n sigma i. sigma i+1+1/4j Sigma i=1n sigma i. sigma 1+2 n sigma i. sigma 1+2 (j>O, N even, N+1 identical to 1, N+2 identical to 2) can be given explicitly in terms of spin eigen-functions. Various properties of the ground state are studied. An approximate determination of the low-lying excited states shows the usual linear frequency-wave-vector relationship.}
}

-->

# References

<a name="cite-Majumdar1970"/><sup>[^](#ref-1) [^](#ref-2) </sup>Majumdar, C, K . 1970. _Antiferromagnetic model with known ground state_. [URL](https://doi.org/10.1088/0022-3719/3/4/019)

